# 从 DFT 到 MD｜超详细「深度势能」材料计算上手指南（下）

<a href="https://bohrium.dp.tech/notebook/fe3921193edc46ebbc51494665ba79b1" target="_blank"><img src="https://cdn.dp.tech/bohrium/web/static/images/open-in-bohrium.svg" alt="Open In Bohrium"/></a>


In [4]:
# Copyright 2023 @ DP Technology
#
# 作者: Wenshuo Liang [liangws@dp.tech]
# 创建日期: 2023-04-13
# 演绎: Haohui Que [quehaohui@dp.tech]
# 最后一次修改时间: 2023-04-23 
#
# 共享协议: 本作品采用[知识共享署名-非商业性使用-相同方式共享 4.0 国际许可协议]进行许可。
# 你可以在该网站获得该协议的具体信息：https://creativecommons.org/licenses/by-nc-sa/4.0/

<span style="color:purple">**这是目前最完整地覆盖从第一性原理计算到分子动力学模拟的超详细「深度势能」材料计算上手指南，主要面向需要入门深度势能材料计算的零基础用户。**</span>

或者你仅需要 10 分钟快速了解 DeePMD-kit，可戳 👉 [DeePMD-kit｜快速开始](https://bohrium.dp.tech/notebook/759614d6c6314dee84a127f11dfc4723)

本教程可在 [Bohrium Notebook](https://bohrium-doc.dp.tech/docs/userguide/Notebook) 上直接运行。**想试试新的想法？不要犹豫，你可以快速将本教程保存至自己的 Bohrium 数据盘，直接进行交互式练习。不需要再困扰于如何从头开始安装所需软件。**

在 Bohrium Notebook 界面，你可以点击界面上方蓝色按钮 `开始连接`，选择 `abacus:3.2.1-deepmd` 镜像及任何一款节点配置，稍等片刻即可运行。

如你遇到任何问题，请联系 [bohrium@dp.tech](mailto:bohrium@dp.tech) 。

<div style="width:auto; height:2px; background:linear-gradient(244deg,rgba(0,0,0,0) 0%,rgba(0,0,0,0.5) 50%,rgba(0,0,0,1) 100%)"></div>



## 背景

在本教程中，我们将以 LiCl 熔体为例，详细介绍深度势能（Deep Potential）模型的训练和应用。深度势能模型是一种基于机器学习的方法，用于精确描述原子间相互作用的势能面。通过训练这种模型，我们可以高效地模拟材料的结构、动力学和热力学性质。具体包括以下示例：

* 章节 1: LAMMPS 经典分子动力学模拟
* 章节 2: ABACUS 第一性原理计算
* 章节 3: DeePMD-kit 深度势能模型训练
* 章节 4: DP-GEN 构建训练数据并生成深度势模型
* 章节 5: LAMMPS 深度势能分子动力学研究

根据内容间的衔接，章节 1\~3 在上篇中进行介绍，章节 4\~5 在下篇中进行介绍。

总的来说，通过结合经典分子动力学模拟（如 LAMMPS）、第一性原理计算（如 ABACUS）和深度势能模型构建（DeePMD-kit 和 DP-GEN），我们可以多角度地研究 LiCl 熔体的性质。这些方法和工具的组合为研究复杂材料系统提供了强大的支持，帮助我们更好地理解材料在不同条件下的性能。

### 开始之前

我们很热切地希望与你分享我们的知识，但是在现在：

**你不需要了解所有事（至少目前是）。** 你的目标是从头到尾完成本教程并获得结果。你不需要在第一次尝试时就了解所有内容。边学习边写下你的问题。使用丰富的 API 文档来了解你正在使用的所有功能。

**你不需要知道算法是如何工作的。** 了解各种计算模拟的优点和局限性以及如何配置材料计算参数非常重要。但是学习可以稍后进行。你需要在很长一段时间内慢慢建立这种计算模拟知识，并且你可以阅读大量教程来复习材料计算项目的步骤。今天，首先要熟悉这个平台。

了解以下基础概念可以帮助你更快速地理解本文：

* 分子动力学 (Molecular Dynamics, MD)，如果你尚未了解，推荐阅读：
    * [5 分钟快速了解分子动力学]()
* 密度泛函理论 (Density Fuctional Theory，DFT)，如果你尚未了解，推荐阅读：
    * [5 分钟快速了解密度泛函理论]()

## 章节 4：DP-GEN 构建训练数据并生成深度势模型

为了构建高质量的LiCl熔体深度势能模型的训练数据集，我们可以使用 DP-GEN(Deep Potential GENerator) 软件。DP-GEN 是一个自动化的训练数据集生成工具，可以根据预设的参数和条件搜索材料的结构和相态空间。

本教程采用DeePMD-kit(2.2.1),ABACUS(3.2.0)和DP-GEN(0.11.0)程序完成。

### 本章节目标：

在学习本章节后，你将能够：

- 掌握 DP-GEN 输入文件（param.json 和 machine.json） 中主要关键词的设置；
- 利用 DP-GEN 为 LiCl 熔体构建训练数据并生成深度势模型；
- 分析和解读 DP-GEN 迭代过程中的结果和输出文件，以便更好地理解模型训练的过程和效果。


### 4.1 下载教程资源

在本教程中，我们以 LiCl 熔体分子为例，构建训练数据并生成深度势模型。我们已经在 *LiCl_DP_Tutorial_Example/chapter4* 中准备了需要的文件。

In [5]:
! if ! [ -e LiCl_DP_Tutorial_Example]; then wget https://bohrium-example.oss-cn-zhangjiakou.aliyuncs.com/notebook/LiCl_DP_Tutorial/LiCl_DP_Tutorial_Example.zip && unzip LiCl_DP_Tutorial_Example.zip; fi;

/bin/bash: line 1: [: missing `]'
--2023-04-24 20:57:55--  https://bohrium-example.oss-cn-zhangjiakou.aliyuncs.com/notebook/LiCl_DP_Tutorial/LiCl_DP_Tutorial_Example.zip
Resolving bohrium-example.oss-cn-zhangjiakou.aliyuncs.com (bohrium-example.oss-cn-zhangjiakou.aliyuncs.com)... 47.92.18.28
Connecting to bohrium-example.oss-cn-zhangjiakou.aliyuncs.com (bohrium-example.oss-cn-zhangjiakou.aliyuncs.com)|47.92.18.28|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12588393 (12M) [application/zip]
Saving to: ‘LiCl_DP_Tutorial_Example.zip.1’

LiCl_DP_Tutorial_Ex 100%[===================>]  12.00M  27.5MB/s    in 0.4s    

2023-04-24 20:57:55 (27.5 MB/s) - ‘LiCl_DP_Tutorial_Example.zip.1’ saved [12588393/12588393]

Archive:  LiCl_DP_Tutorial_Example.zip
replace LiCl_DP_Tutorial_Example/chapter1/msd.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [6]:
! tree LiCl_DP_Tutorial_Example/chapter4 -L 2

LiCl_DP_Tutorial_Example/chapter4
├── abacus
│   ├── Cl_ONCV_PBE-1.2.upf
│   ├── Cl_gga_8au_100Ry_2s2p1d.orb
│   ├── Li_ONCV_PBE-1.2.upf
│   └── Li_gga_8au_100Ry_4s1p.orb
├── machine.json
├── max-devi-f.png
├── max_devi_f.py
└── param_abacus.json

1 directory, 8 files


- `*.upf` 和 `*.orb` 是 ABACUS 的输入文件
- `param.json` 是运行当前任务的 DP-GEN 设置。
- `machine.json` 是一个任务调度程序，其中设置了计算机环境和资源要求。

### 4.2 DP-GEN 输入文件

DP-GEN 运行过程包含一系列连续迭代，按顺序进行，例如将系统加热到特定温度。每次迭代由三个步骤组成：探索 (Exploration)，标记 (Labeling)和训练 (Training)。

首先，介绍 DP-GEN 运行过程所需的输入文件。


#### param.json

`param.json` 中的关键字可以分为 4 个部分：

- 系统和数据 (System and Data)：用于指定原子类型、初始数据等。
- 训练 (Training)：主要用于指定训练步骤中的任务;
- 探索 (Exploration)：主要用于在探索步骤中指定任务;
- 标记 (Labeling)：主要用于指定标记步骤中的任务。

这里我们以LiCl熔体为例，介绍`param.json`中的主要关键词。

**系统和数据 (System and Data)**

系统和数据相关内容：

```
{
     "type_map": ["Li","Cl"],
     "mass_map": [6.941,35.453],
     "init_data_prefix": "../ex3",
     "init_data_sys": ["00.data/training_data"],
     "sys_format": "abacus/stru",
     "sys_configs_prefix": "../ex2",
     "sys_configs": [["01.md/STRU"]],
     "_comment": " that's all ",

```

关键词描述：

| 键词                  | 字段类型      | 描述                                                     |
|-----------------------|--------------|----------------------------------------------------------|
| "type_map"            | list         | 元素列表，这里是Li和Cl                                    |
| "mass_map"            | list         | 原子质量列表                                              |
| "init_data_prefix"    | str          | initial data 的前置路径                                   |
| "init_data_sys"       | list         | 初始训练数据文件的路径列表。可以使用绝对路径或相对路径        |
| "sys_format"	        | str          | 指定构型的格式                                            |
| "sys_configs_prefix"  | str          | sys_configs 的前置路径                                    |
| "sys_configs"         | list         | 构型文件的路径列表，此处支持通配符                          |

案例说明：

“type_map”和“mass_map”给出了元素类型和原子质量。在这里，系统包含锂（Li）和氯（Cl）两种，质量分别为6.941和35.453。
“init_data_prefix”和“init_data_sys”关键词共同指定了初始训练数据的位置。“sys_configs_prefix”和“sys_configs”共同指定了探索的构型的位置。“sys_format”指定了构型的格式。在这里，训练数据位于../ex3/00.data/training_data目录下。构型文件位于../ex2/01.md/STRU目录下，采用ABACUS软件的abacus/stru格式


**训练(Training)**

与训练相关的内容如下：

```
     "numb_models": 4,
     "default_training_param": {
         "model": {
             "type_map": ["Li","Cl"],
             "descriptor": {
                 "type": "se_e2_a",
                 "sel": [128,128],                
                 "rcut_smth": 0.5,
                 "rcut": 7.0,
                 "neuron": [20,40,80],
                 "resnet_dt": false,
                 "axis_neuron": 12,
                 "seed": 1
             },
             "fitting_net": {
                 "neuron": [200,200,200],
                 "resnet_dt": true,
                 "seed": 1
             }
         },
         "learning_rate": {
             "type": "exp",
             "start_lr": 0.001,
             "decay_steps": 5000
         },
         "loss": {
             "start_pref_e": 0.02,
             "limit_pref_e": 1,
             "start_pref_f": 1000,
             "limit_pref_f": 1,
             "start_pref_v": 0,
             "limit_pref_v": 0
         },
         "training": {
             "numb_steps": 400000,
             "disp_file": "lcurve.out",
             "disp_freq": 1000,
             "numb_test": 1,
             "save_freq": 10000,
             "save_ckpt": "model.ckpt",
             "disp_training": true,
             "time_training": true,
             "profiling": false,
             "profiling_file": "timeline.json",
             "_comment": "that's all"
         }
     },
```

关键词描述：

| 键词                      | 字段类型  | 描述                          |
|---------------------------|----------|-------------------------------|
| "numb_models"             | int      | 在 00.train 中训练的模型数量。  |
| "default_training_param"  | dict     | DeePMD-kit 的训练参数          |

案例说明：

训练相关键指定训练任务的详细信息。`numb_models`指定要训练的模型数量。`default_training_param`指定了 DeePMD-kit 的训练参数。在这里，将训练 4 个 DP 模型。

DP-GEN 的训练部分由 DeePMD-kit 执行，因此此处的关键字与 DeePMD-kit 的关键字相同，此处不再赘述。有关这些关键字的详细说明，请访问[DeePMD-kit 文档](https://deepmd.readthedocs.io/)。

**探索 (Exploration) **

与探索相关的内容如下：

```
     "model_devi_dt": 0.001,
     "model_devi_skip": 0,
     "model_devi_f_trust_lo": 0.08,
     "model_devi_f_trust_hi": 0.18,
     "model_devi_merge_traj": true,
     "model_devi_clean_traj": false,
     "model_devi_jobs":  [
        {"sys_idx": [0],"temps": [900,1000,1100,1200],"press": [0,10,100,1000,10000], "trj_freq": 10, "nsteps": 100000,"ensemble": "npt", "_idx": "00"},
        {"sys_idx": [0],"temps": [900,1000,1100,1200],"press": [0,10,100,1000,10000], "trj_freq": 10, "nsteps": 100000,"ensemble": "npt", "_idx": "01"},
        {"sys_idx": [0],"temps": [900,1000,1100,1200],"press": [0,10,100,1000,10000], "trj_freq": 10, "nsteps": 100000,"ensemble": "npt", "_idx": "02"}   
     ],
```

关键词描述：

| 键词                      | 字段类型                    | 描述   |
|--------------------------|-------------------------|---------------|
| "model_devi_dt"          | float  | MD 的时间步长                                                                                                                                                                                                                                |
| "model_devi_skip"        | int    | 每个 MD 中为 fp 跳过的结构数                                                                                                                                                                                                |
| "model_devi_f_trust_lo"  | float  | 选择的力下限。如果为 List，则应分别为每个索引设置sys_configs。                                                                                                                                 |
| "model_devi_f_trust_hi"  | int    | 选择的力上限。如果为 List，则应分别为每个索引设置sys_configs。                                                                                                                                  |                                                                                         |
| "model_devi_clean_traj"  | bool or int    | 如果model_devi_clean_traj的类型是布尔类型，则表示是否清理MD中的traj文件。如果是 Int 类型，则将保留 traj 文件夹的最新 n 次迭代，其他迭代将被删除。  |
| "model_devi_clean_traj"  | bool           | 控制在模型偏差（model_devi）阶段是否合并生成的轨迹文件|
| "model_devi_jobs"        | list            | 01.model_devi 中的探索设置。列表中的每个字典对应于一次迭代。model_devi_jobs 的索引与迭代的索引完全一致                                               |
| &nbsp;&nbsp;&nbsp;&nbsp;"sys_idx"   | List of integer         | 选择系统作为MD的初始结构并进行探索。序列与“sys_configs”完全对应。 |
| &nbsp;&nbsp;&nbsp;&nbsp;"temps" | list  | 分子动力学模拟的温度 (K)
| &nbsp;&nbsp;&nbsp;&nbsp;"press" | list  | 分子动力学模拟的压力 (Bar) 
| &nbsp;&nbsp;&nbsp;&nbsp;"trj_freq"   | int          | MD中轨迹的保存频率。                  |
| &nbsp;&nbsp;&nbsp;&nbsp;"nsteps"     | int          | 分子动力学运行步数                                 |
| &nbsp;&nbsp;&nbsp;&nbsp;"ensembles"  | str          | 决定在 MD 中选择的集成算法，选项包括 “npt” ， “nvt”等. |

案例说明

在在“model_devi_jobs”中设置了三次迭代。对于每次迭代，在不同的温度（900, 1000, 1100和1200 K）和压力条件（0, 0.1, 1, 10和100 GPa）下，使用npt系综和“sys_configs_prefix”和“sys_configs”指定的构型进行100000步模拟，时间步长为0.001 ps。我们选择保存 MD 轨迹文件，并将保存频率“trj_freq”设置为 10。如果轨迹中构型的“max_devi_f”介于 0.08 和 0.18 之间，DP-GEN 会将该结构视为候选结构。如果要保存 traj 文件夹的最近n次迭代，可以将“model_devi_clean_traj”设置为整数。

** 标记 (Labeling)**

与标记相关的内容如下：

```
    "fp_style": "abacus",
    "shuffle_poscar": false,
    "fp_task_max": 200,
    "fp_task_min": 50,
    "fp_pp_path": "./abacus",
    "fp_pp_files": ["Li_ONCV_PBE-1.2.upf","Cl_ONCV_PBE-1.2.upf"],
    "fp_orb_files": ["Li_gga_8au_100Ry_4s1p.orb","Cl_gga_8au_100Ry_2s2p1d.orb"],
    "k_points":[1, 1, 1, 0, 0, 0],
    "user_fp_params":{
    "ntype": 2, 
    "symmetry": 0,
    "vdw_method":"d3_bj",
    "ecutwfc": 100,
    "scf_thr":1e-7,
    "scf_nmax":120, 
    "basis_type":"lcao", 
    "smearing_method": "gauss",
    "smearing_sigma": 0.002,
    "mixing_type": "pulay",
    "mixing_beta": 0.4,
    "cal_force":1,
    "cal_stress":1
```

关键词描述：

| 键词               | 字段类型            | 描述                                                                                                              |
|-------------------|-----------------|---------------------------------------------------------------------|
| "fp_style"        | String          | 第一性原理软件软件。到目前为止，选项包括ABACUS, VASP等。                |
| "shuffle_poscar"  | Boolean         |                                                                     |
| "fp_task_max"     | Integer         | 每次迭代 在 02.fp 中要计算的最大结构。                                 |
| "fp_task_min"     | Integer         | 每次迭代 在 02.fp 中要计算的最小结构。                                 |
| "fp_pp_path"      | String          | 用于 02.fp 的赝势文件路径。                                           |
| "fp_pp_files"     | List of string  | 用于 02.fp 的赝势文件。请注意，元素的顺序应与 type_map 中的顺序相对应。  |
| "fp_orb_files"    | List of string  | 用于 02.fp 的轨道文件。请注意，元素的顺序应与 type_map 中的顺序相对应。  |
| "k_points"        | List of Integer | 用于生成ABACUS KPT文件。                                                    |
| "user_fp_params"  | dict            | 用于生成ABACUS INPUT文件。如果"user_fp_params"中指定了 kspacing，可以不设置"k_points"。   |

案例说明：

标记相关键词指定标记任务的详细信息。 在这里，最少 50 个和最多 200 个结构将使用 ABACUS 代码进行标记，在每次迭代中，INPUT 文件依据“user_fp_params”生成，KPT文件依据 “k_points”生成。请注意，"fp_pp_files" 和 "fp_orb_files" 中元素的顺序应与 `type_map` 中的顺序相对应。

#### machine.json

DP-GEN 运行过程中的每次迭代都由三个步骤组成：探索、标注和训练。因此，machine.json 由三个步骤组成：**train**、**model_devi** 和 **fp**。每个步骤都是字典列表。每个字典都可以被视为一个独立的计算环境。

在本节中，我们将向您展示如何在Bohrium上执行`train`, `model_devi`和`fp` 步骤。 对于每个步骤，需要三种类型的关键词：

- 命令 (Command)：提供用于执行每个步骤的命令。
- 机器 (Machine)：指定机器环境（本地工作站、本地或远程集群或云服务器）。
- 资源 (Resources)：指定组、节点、CPU 和 GPU 的数量;启用虚拟环境。


在此示例中，我们在Bohrium上执行`train`步骤。

```json
{
  "api_version": "1.0",
  "deepmd_version": "2.1.5",
  "train" :[
    {
      "command": "dp",
      "machine": {
        "batch_type": "Lebesgue",
        "context_type": "LebesgueContext",
        "local_root" : "./",
        "remote_profile":{
          "email": "xxx", 
          "password": "xxx", 
          "program_id": xxx,
            "keep_backup":true,
            "input_data":{
                "job_type": "container",
                "log_file": "00*/train.log",
                "grouped":true,
                "job_name": "dpgen_train_job",
                "disk_size": 100,
                "scass_type":"c12_m92_1 * NVIDIA V100",
                "checkpoint_files":["00*/checkpoint","00*/model.ckpt*"],
                "checkpoint_time":5,
                "platform": "ali",
                "image_name":"registry.dp.tech/dptech/deepmd-kit:2.1.5-cuda11.6",
                "on_demand":0
            }
        }
      },
      "resources": {
        "local_root":"./",
        "group_size": 1
      }
    }],
  "model_devi":
    [{
      "command": "export LAMMPS_PLUGIN_PATH=/opt/deepmd-kit-2.1.5/lib/deepmd_lmp && lmp -i input.lammps -v restart 0",
      "machine": {
        "batch_type": "Lebesgue",
        "context_type": "LebesgueContext",
        "local_root" : "./",
        "remote_profile":{
          "email": "xxx", 
          "password": "xxx", 
          "program_id": xxx,
            "keep_backup":true,
            "input_data":{
              "job_type": "container",
              "log_file": "task*/model_devi.out",
              "grouped":true,
              "job_name": "dpgen_model_devi_job",
              "disk_size": 200,
              "scass_type":"c12_m92_1 * NVIDIA V100",
              "platform": "ali",
              "image_name":"registry.dp.tech/dptech/deepmd-kit:2.1.5-cuda11.6",
              "checkpoint_files": "sync_files",
              "checkpoint_time":5,
              "on_demand":0
            }
        }
      },
      "resources": {
        "local_root":"./",
        "group_size": 1
      }
    }],
  "fp":
    [{
      "command": "OMP_NUM_THREADS=1 mpirun -np 4 abacus",
      "machine": {
        "batch_type": "Lebesgue",
        "context_type": "LebesgueContext",
        "local_root" : "./",
        "remote_profile":{
          "email": "xxx", 
          "password": "xxx", 
          "program_id": xxx,
            "keep_backup":true,
            "input_data":{
              "log_file": "task*/output",
              "grouped":true,
              "job_name": "dpgen_fp_job",
              "checkpoint_files": "sync_files",
              "checkpoint_time":5,
              "scass_type":"c8_m64_cpu",
              "platform": "ali",
              "image_name":"registry.dp.tech/dptech/abacus:3.1.0",
              "job_type": "container",
              "on_demand":0
            }
        }
      },
      "resources": {
        "group_size": 2,
        "local_root":"./",
        "source_list": []
      }
    }
  ]
}
```

案例说明：

在 command 参数中，`train`，`model_devi`和`fp`步骤使用的程序分别为 DeePMD-kit，LAMMPS 和 ABACUS，其相应的调用命令分别为“dp”，"export LAMMPS_PLUGIN_PATH=/opt/deepmd-kit-2.1.5/lib/deepmd_lmp && lmp -i input.lammps -v restart 0" 和 "OMP_NUM_THREADS=1 mpirun -np 4 abacus"。

在 machine 参数中，"scass_type"指定训练使用的机型。对于`train`和`model_devi`步骤，建议使用GPU机型，这里使用的机型为“c12_m92_1 * NVIDIA V100”。对于`fp`步骤，建议使用CPU机型，这里使用的机型为“c8_m64_cpu”。

在 resources 参数中，“group_size”指定了一个 group 中任务的数量。对于`train`和`model_devi`步骤，由于任务数量较少（分别为4和20），我们可以将“group_size”设置为 1, 使所有任务同时执行。对于`fp`步骤，由于任务数量较多（200），我们可以将“group_size”设置为 2, 使100个任务同时执行。

注意：用户需要填入自己的 Bohrium 账户邮箱，密码和项目ID（共三处）。其他参数通常不需修改。

### 4.3 运行 运行DP-GEN

准备好了 `param.json` 和 `machine.json`，我们就可以通过以下方式轻松运行 DP-GEN：

In [7]:
! cd LiCl_DP_Tutorial_Example/chapter4 && dpgen run param_abacus.json machine.json

/bin/bash: line 1: dpgen: command not found


成功运行 DP-GEN 之后，在 LiCl_DP_Tutorial_Example/chapter4 下，可以发现生成了一个文件夹和两个文件：

- iter.000000 文件夹：包含 DP-GEN 迭代过程中第一次迭代的主要结果；
- dpgen.log 文件：主要记录时间和迭代信息；
- record.dpgen 文件：记录迭代过程的当前阶段。

如果 DP-GEN 的进程由于某种原因停止，DP-GEN 将通过 `record.dpgen`文件自动恢复主进程。我们也可以根据自己的目的手动更改它，例如删除上次迭代并从一个检查点恢复。`record.dpgen`文件每行包含两个数字：第一个是迭代的索引，第二个是 0 到 9 之间的数字，记录每个迭代中的哪个阶段当前正在运行。

| Index of iterations  | Stage in each iteration     | Process          |
|----------------------|-----------------------------|------------------|
| 0                    | 0                           | make_train       |
| 0                    | 1                           | run_train        |
| 0                    | 2                           | post_train       |
| 0                    | 3                           | make_model_devi  |
| 0                    | 4                           | run_model_devi   |
| 0                    | 5                           | post_model_devi  |
| 0                    | 6                           | make_fp          |
| 0                    | 7                           | run_fp           |
| 0                    | 8                           | post_fp          |

### 4.4 结果分析

第一次迭代完成后，iter.000000 的文件夹结构如下，主要是 3 个文件夹:

In [8]:
! cd LiCl_DP_Tutorial_Example/chapter4 && tree iter.000000/ -L 1 

iter.000000/  [error opening dir]

0 directories, 0 files


- 00.train 文件夹：主要是基于现有数据训练的多个 DP 模型（默认是 4 个）；
- 01.model_devi 文件夹：使用 00.train 中得到的 DP 模型进行 MD 模拟，产生新的构型；
- 02.fp 文件夹：对选定的构型进行第一原理计算，并将计算结果转换成训练数据


首先，我们来查看 `iter.000000`/ `00.train`。

In [ ]:
! cd LiCl_DP_Tutorial_Example/chapter4 && tree iter.000000/00.train -L 1

- 文件夹 00x 包含 DeePMD-kit 的输入和输出文件，其中训练了模型。
- graph.00x.pb ，链接到 00x/frozen.pb，是 DeePMD-kit 生成的模型。这些模型之间的唯一区别是神经网络初始化的随机种子。

让我们随机选择其中之一查看，例如 000。

In [ ]:
! cd LiCl_DP_Tutorial_Example/chapter4 && tree iter.000000/00.train/000 -L 1

- `input.json` 是当前任务的 DeePMD-kit 的设置。
- `checkpoint`用于重新开始训练。
- `model.ckpt*` 是与模型相关的文件。
- `frozen_model.pb`是冻结模型。
- `lcurve.out`记录能量和力的训练精度。
- `train.log`包括版本、数据、硬件信息、时间等。


然后，我们来查看`iter.000000/ 01.model_devi`。

In [ ]:
! cd LiCl_DP_Tutorial_Example/chapter4 && tree iter.000000/01.model_devi -L 1

- 文件夹 confs 包含从您在 param.json 的“sys_configs”中设置的 STRU 转换而来的 LAMMPS MD 的初始配置。
- 文件夹 task.000.00000x 包含 LAMMPS 的输入和输出文件。我们可以随机选择其中之一，例如 task.000.000000。

In [ ]:
! cd LiCl_DP_Tutorial_Example/chapter4 && tree iter.000000/01.model_devi/task.000.000001

- `conf.lmp`，链接到文件夹 confs 中的“000.0001.lmp”，作为 MD 的初始配置。
- `input.lammps` 是 LAMMPS 的输入文件。
- `model_devi.out` 记录在 MD 训练中相关的标签，能量和力的模型偏差。它是选择结构和进行第一性原理计算的标准。
- `traj` 存储LAMMPS MD轨迹。

通过查看 `model_devi.out` 的前几行, 您会看到:

In [ ]:
! cd LiCl_DP_Tutorial_Example/chapter4 && head -n 5 ./iter.000000/01.model_devi/task.000.000001/model_devi.out

想一下，我们将“trj_freq”设置为10，因此每10个步骤保存结构。是否选择结构取决于其`“max_devi_f”`。如果它介于`“model_devi_f_trust_lo”`（0.08）和`“model_devi_f_trust_hi”`（0.18）之间，DP-GEN 会将该结构视为候选结构。对于0，10，20 和 30 步保存结构，其`“max_devi_f”`均小于 0.08，所以不会被视为候选结构。


最后，我们来查看 `iter.000000/ 02.fp` 。

In [ ]:
! cd LiCl_DP_Tutorial_Example/chapter4 && tree iter.000000/01.model_devi/task.000.000001